In [120]:
import genanki
import csv
import argparse
import os
import hashlib
import time
from pathlib import Path
import logging

from src.anki import read_file, make_deck, make_model, make_notes, make_package, read_package
from src.media import add_images, add_phonetics, add_sounds
from src.log_config import configure_logging

configure_logging(ignore_libs=["PIL", "phonemizer"])

INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
IMG_DIR = MEDIA_DIR / "img"
AUDIO_DIR = MEDIA_DIR / "sound"
for d in [INPUT_DIR, OUTPUT_DIR, TEMPLATE_DIR, MEDIA_DIR, IMG_DIR, AUDIO_DIR]:
    d.mkdir(parents=True, exist_ok=True)


superdeck_name = "Français"
file_name = "Les_nombres"
file_name = "L'alphabet"
file_name = "Cours_01"
file_name = "Cours_01_QA"
file_name = "Cours_02"
# file_name = "Cours_03"
deck_name = superdeck_name + "::" + file_name
file_type = ".md"
df_file = INPUT_DIR / (file_name + file_type)
apkg_file = OUTPUT_DIR / (file_name + ".apkg")
style = TEMPLATE_DIR / "style.css"
img_files, sound_files = [],[]


In [121]:
df = read_file(df_file)
df = add_phonetics(df)
df, img_files = add_images(df, IMG_DIR, force_replace=False)
# df, sound_files = add_sounds(df, AUDIO_DIR, force_replace=False)
media_files = img_files + sound_files
deck = make_deck(df, deck_name)
model = make_model(df, style)
make_notes(deck, model, df)
make_package(deck, media_files=img_files, apkg_file=apkg_file)
df.head()

[22:30:03] [INFO] [media] - Added phonetics for 35 words.
[22:30:03] [INFO] [media] - Added 35 images for 35 words.
[22:30:03] [INFO] [anki] - Front-to-back and back-to-front cards will be created.
[22:30:03] [INFO] [anki] - Created 35 notes out of 35 possible lines.


,Front,Back,Extra,More,Phonetics,Image,Audio,Reverse
0,le jour,der Tag,,,/lə- ʒˈuʁ/,"<img src=""img_badffde5fef4137a.png"">",,
1,la semaine,die Woche,,,/la- səmˈɛn/,"<img src=""img_1387874aa21491e5.png"">",,
2,la semaine dernière,letzte Woche,,,/la- səmˈɛn dɛʁnjˈɛʁ/,"<img src=""img_ff13f511caf0e865.png"">",,
3,fumer,rauchen,,,/fymˈe/,"<img src=""img_3349713c4db68bab.png"">",,
4,un jeu de chance,ein Glückspiel,,,/œ̃ ʒˈø də- ʃˈɑ̃s/,"<img src=""img_bb87177e976487c8.png"">",,


In [122]:
import os
from pathlib import Path
from phonemizer.backend import EspeakBackend
from phonemizer.backend.espeak.wrapper import EspeakWrapper
# Set up espeak functio
if os.uname().sysname == 'Darwin':
    EspeakWrapper.set_library(Path("/opt/local/bin/espeak")) # macports version

language = "fr-fr"
backend = EspeakBackend(language=language, with_stress=True, preserve_punctuation=True)
phonemize_it = lambda x: f"{backend.phonemize([str(x)], strip=True)[0]}"
# phonemize_it("quatre-vingts, quatre-vingt-un")

In [123]:
words = [
    "je m'appelle",
    "tu t'appelles",
    "il, elle s'appelle",
    "nous nous appelons",
    "vous vous appelez",
    "ils, elles s'appellent"
]

words = "le pommé, les pommes"
words = """le croissant, les croissants.
la pomme, les pommes.
l'enfant, les enfants."""

words = """je suis.
tu es.
il, elle est.
nous sommes.
vous êtes.
ils, elles sont."""

words = """j'ai.
tu as.
il, elle a.
nous avons.
vous avez.
ils, elles ont."""

words = """faire.
je fais.
tu fais.
il, elle fait.
nous faisons.
vous faites.
ils, elles font."""

phonemize_it(words)


'fˈɛʁ.\nʒə- fˈɛ.\nty fˈɛ.\nˈil, ɛl fˈɛ.\nnu fəzˈɔ̃.\nvu fˈɛt.\nˈil, ɛl fˈɔ̃.'

In [129]:
def generate_unique_id(string):
    """Create a unique ID based on a string."""
    hash_object = hashlib.sha1(string.encode())
    return int(hash_object.hexdigest(), 16) % (10 ** 10)

def hash_str(input_str):
    """Generates a 16-character SHA-256 hash of the input string."""
    try:
        encoded_string = input_str.encode()
        sha256_hash = hashlib.sha256()
        sha256_hash.update(encoded_string)
        return sha256_hash.hexdigest()[:16]
    except AttributeError:
        raise ValueError("Input must be a string")

In [130]:
generate_unique_id("Hello"), hash_str("Hello")

(6440359920, '185f8db32271fe25')